---
Stellen Sie bitte sicher, dass alles wie vorhergesehen läuft, bevor Sie dieses Übungsblatt abgeben. **Starten Sie den Kernel neu** (in der Menüleiste die Option Kernel$\rightarrow$Restart auswählen) und **validieren** Sie anschließend das Übungsblatt (in der Menüleiste auf Validate klicken) um Rückmeldung zu eventuellen fehlenden oder fehlerhaften Eingaben zu erhalten. 

Füllen Sie alle Stellen im Übungsblatt aus, welche entweder `DEIN CODE HIER` oder "DEINE ANTWORT HIER" enthalten. Geben Sie unterhalb Ihren vollständigen Namen an.

Wenn Sie Code-Bestandteile aus anderen Quellen (wie z.B. Stackoverflow) kopieren, dann machen sie den kopierten Code in ihrer Quellcodedatei kenntlich und fügen eine Referenz auf die Quelle als Kommentar hinzu.

Wenn Sie die Aufgaben in einer Gruppe erledigen, dann fügen Sie die Namen aller Gruppenmitglieder in der nachfolgende Zelle zu `Name` und zusätzlich als Kommentar am Anfang Ihrer Quellcodedatei hinzu.

---

In [ ]:
NAME = "Nicolas Bernhart"

---

<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="width: 240px"><img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo"></td>
        <td style="width: 100%">
            <div style="text-align:right; width: 100%; text-align:right"><font style="font-size:38px"><b>Methoden der Softwareentwicklung II</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>SS 2024</b></font></div>
        </td>
    </tr>
</table>

----
# Übungsblatt 5

In [ ]:
import jagl
import os
import shutil

------
Abgabe bis <b>Sonntag, 05. Mai 2024, 23:55 Uhr</b>

---

### Achtung:

*Bitte speichern Sie Ihre Quellcode-Dateien (&ast;.c;&ast;.cpp;&ast;.h;...) im selben Ordner wie dieses Übungsblatt. Beachten Sie außerdem die Kommentare zur richtigen Benennung Ihrer Dateien. Nur so ist eine korrekte Abgabe und Auswertung Ihrer Lösungen möglich.*


*Bitte verwenden Sie für Ihre Lösungen einen ordentlichen Programmierstil. Vor allem auf eine korrekte Formatierung, aussagekräftige Variablennamen und ausführliche Kommentare sollte dabei Wert gelegt werden.*


## Methoden der Spieleentwicklung V (100%)

Sie haben den letzte Woche implementierten Prototypen Ihres Spiels nun einer größeren Gruppe von Spieletester:innen zum Testen gegeben. Deren Feedback zeigte vor allem zwei Kritikpunkte auf: (1) Das Inventar ist viel zu klein, und (2) es kommt immer wieder zu Abstürzen, die wahrscheinlich von Memory Leaks verursacht werden. Daher haben Sie sich entschlossen, für den nächsten Prototpyen diese zwei Punkte anzugehen.


### Unendliches Inventar

Ihre Spieletester:innen haben sich darüber beschwert, dass das Inventar zu beschränkt ist und sie viel zu viel [Loot](https://de.wikipedia.org/wiki/Loot_(Computerspiel)) in den Dungeons zurücklassen müssen. Es wurde daher beschlossen, dass alle Charaktere nun ein unbegrenztes Inventar bekommen sollen (Die Ausrüstung der Heldin ist davon nicht betroffen). Dies soll durch die Verwendung eines geeigneten Datencontainers der Standardbibliothek umgesetzt werden. Schauen Sie sich die im Webinar genannten Container genauer an und wählen Sie einen passenden Container aus.

Damit der restliche Code möglichst unverändert weiterverwendet werden kann, soll die dazugehörige öffentliche Schnittstelle (die Funktionen `addInventarItem()`, `removeInventarItem()`, `getInventory()` und `sellItem()`) dabei nicht geändert werden (abgesehen von den Änderungen, die sich durch die weitere Aufgabenstellung unten ergeben). Es sollen weiterhin Exceptions geworfen werden, wenn auf einen ungültigen Slot zugegriffen wird.


### Object Ownership


#### Charaktere und die Klasse `Game`

Ein weiterer Punkt, den die Spieletester:innen herausgefunden haben, ist, dass das Spiel nach einer längeren Spielzeit abstürzt, da wohl ein Memory Leak vorliegt. Aus diesen Grund soll die Frage der Object Ownership für die Charaktere und Gegenstände gelöst werden. Der Plan ist die Implementierung einer eigenen Klasse `Game`, der die Ownership über alle Charaktere übernimmt. Für die Charaktere sollen ab jetzt ausschließlich Heap-Objekte (also mit `new` erzeugte Objekte) zum Einsatz kommen. Diese sollen als Key-Value Paare gespeichert werden, wobei der Name des Charakters den Key bildet (ab jetzt müssen alle Charakternamen eindeutig sein).Schauen Sie sich die im Webinar genannten Container genauer an und wählen Sie einen passenden Container aus. Ob Sie Smart Pointer verwenden wollen, bleibt Ihnen überlassen.

Diese Klasse soll außerdem die Objektfunktion `play()` besitzen, die das eigentliche Spiel implementiert (Der Code, der vorher in der main()-Funktion zu finden war, wandert nun in diese Funktion).

Die Klasse `Game` soll auch sicherstellen, dass alle Charaktere, deren Lebenspunkte unter 0 sinken, von der Klasse `Game` direkt nach dem Looten zerstört werden. Damit sollte das Problem der Spieleabstürze gelöst sein. Spätestens im Destruktor von `Game` sollten alle noch lebenden Charaktere zerstört werden. Implementieren Sie entsprechende Debuggingausgaben in den Destruktoren aller Charaktere, damit Sie nachverfolgen können, dass am Ende auch tatsächlich alle Charaktere zerstört wurden.


#### Gegenstände

Auch für die Gegenstände muss die Ownership-Frage gestellt werden. Für Gegenstände, die sich im Iventar eines Charakters befinden, übernimmt der Charakter die Ownership. 

Zur Vereinfachung sollen für die Verwaltung der Gegenstände Smart Pointer verwendet werden. Damit soll sichergestellt werden, dass mittels Reference Counting Gegenstände automatisch zerstört werden, die nicht mehr benötigt werden. Dadurch ergeben sich folgende Änderungen:


- Das Inventar bzw. das Array `gear` für die Ausrüstung speichert nun Objekte vom Typ `std::shared_ptr<Item>`.


- Funktionen, die Gegenstände zurückgeben, haben nun als Rückgabewert einen Smart Pointer (z.B. `std::shared_ptr<Item> getInventory(int index)`).


- Funktionen, die Gegenstände als Eingabeargument erwarten, bekommen nun einen Smart Pointer übergeben (z.B. `int addInventarItem(std::shared_ptr<Item> item)`).


Gegenstände, die sich nicht im Inventar eines Charakters befinden (z.B. verkaufte Gegenstände), brauchen auch einen Object Owner. In diesem Fall soll auch die Klasse `Game` die Object Ownership übernehmen. Wählen Sie wieder selbstständig einen passenden Datencontainer aus. Die Gegenstände brauchen kein eindeutig identifizierendes Merkmal.

Überprüfen Sie durch entsprechende Debuggingausgaben im Destruktor, ob die Gegenstände auch tatsächlich gelöscht werden.

### Testprogramm

Implementieren Sie wieder ein Testprogramm, diesmal in der `play()` Methode, in dem die Heldin gegen zwei Feinde kämpft, den Zauberer Pascal und den Kämpfer Matthias. Am Ende der Kämpfe sollen wieder alle erbeuteten Gegenstände verkauft werden. In der `main()`-Funktion wird nur eine Instanz der Klasse `Game` als Stackobjekt erzeugt und dann die Methode `play()` aufgerufen.

<div class="alert alert-block alert-info">
<b>Info:</b>  Info: Sollten Sie keine Lösung für Exercise Sheet 4 haben, oder sicher sein wollen, dass Sie eine korrekte Basis für Exercise Sheet 5 verwenden, verwenden Sie bitte die Musterlösung zu Exercise Sheet 4, die am Donnerstag veröffentlicht wird. 
</div>

In [ ]:

# new test suite
jagl.testsuite_begin("Exercise 1")

# delete all build artifacts
_ = jagl.remove_paths(files=["prototyp5", "*.o"])

In [1]:
%%bash
#
# Da das Kompilieren von C++-Programmen bei wachsender Größe die Eingabe langer Befehle erforderlich
# macht, kommen in der Praxis sogenannte Build Management Systeme zum Einsatz. Diese helfen bei der
# Automatisierung des Build-Prozesses und erleichtern Programmierern das Leben.
# Eines der am weitersten verbreiteten Build Management Systeme unter Linux/Unix für C/C++-Programme
# ist make.
#
# Ihre Aufgabe ist es, ein Makefile (das auch den Namen "Makefile" hat!) zu erzeugen, mithilfe welchem Ihr Programm kompiliert wird.
# Studieren Sie sich die Folien aus der ersten Einheit, bzw. die dazugehörigen Unterlagen des
# Selbststudiums, um ein funktionsfähiges Makefile zu erzeugen.
#
# Kopieren Sie vor der Abgabe ihr Makefile und Ihre Quellcodedatein in dasselbe Verzeichnis, 
# in dem dieser Übungszettel zu finden ist.
#
# Der Name der ausführbaren Datei sollte 'prototyp5' lauten.
#

# Das Ausführen dieser Zelle sollte das Makefile ausführen
make

g++ -c -Werror -Wall -o main.o main.cpp
g++ -c -Werror -Wall -o game.o game.cpp
g++ -c -Werror -Wall -o hero.o hero.cpp
g++ -c -Werror -Wall -o character.o character.cpp
g++ -c -Werror -Wall -o item.o item.cpp
g++ -c -Werror -Wall -o npc.o npc.cpp
g++ -c -Werror -Wall -o fighter.o fighter.cpp
g++ -c -Werror -Wall -o sorcerer.o sorcerer.cpp
g++	 -o prototyp5 main.o game.o hero.o character.o item.o npc.o fighter.o sorcerer.o


In [2]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_5_1_solved = True

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_1_1(result, suite, case):
    varname = "exercise_5_1_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        if eval(varname):
            result.setSucceeded("Exercise solved.")
        else:
            result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Check source files", deps=["1"])
def testcase_1_2(result, suite, case):
    result.setSucceeded()
    jagl.check_path_exists(result, files=["Makefile"], message = "Are you sure that you uploaded a Makefile and named it \"Makefile\"?")
    jagl.check_path_exists(result, files=["hero.h", "hero.cpp", "character.h", "character.cpp", "npc.h", "sorcerer.h", "sorcerer.cpp", "fighter.h", "fighter.cpp", "item.h", "item.cpp", "main.cpp"], message = "Did you upload the source files?")
    if result.isSucceeded():
        result.addMessage("Makefile and source files were found.")

In [ ]:
@jagl.testcase("3", desc="Building program", deps=["1"])
def testcase_1_3(result, suite, case):
    progname = "prototyp5"
    if os.path.isfile(progname):
        result.setSucceeded("The program could be build successfully.")
        
    else:
        res = jagl.exec_bin("make")
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(progname):
                    result.setSucceeded("The program could be build successfully.")
                else:
                    result.setFailed("make was executed successfully, but I couldn't find an executable (Did your name your executable '" + progname + "'?).")
                if len(stderr) > 0:
                    result.setFailed(["However, there were errors."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be build:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("make could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling make. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed('Unknown error during execution of make: ' + str(res.error) + " Please inform your course instructor.")

In [ ]:
@jagl.testcase("4", desc="Executing program", deps=["3"])
def testcase_1_4(result, suite, case):
    progname = "prototyp5"
    execresult = jagl.exec_bin("./" + progname, [], timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = True)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")

In [ ]:
# delete all build artifacts
_ = jagl.remove_paths(files=["prototyp5", "*.o"])